In [12]:
import sys
sys.path.append('..')
from api.api import PDTB
import codecs
import json

In [7]:
pdtb3 = PDTB('/home/pengfei/data/pdtb3/all/conll/')
pdtb = pdtb3

In [204]:
output = [json.loads(o) for o in open('/home/pengfei/Github/conll3_discourse/data/output.json').readlines()]


def get_minus_offset(parse_dict, docid, sentid):
    ret=0
    for sent in parse_dict[docid]['sentences'][:sentid]:
        ret += len(sent['words'])
    return ret

def check_if_arg(token_id, sent_id, Arg_token_list):
    return ((sent_id, token_id) in Arg_token_list)


class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    BACKGROUND = '\033[7m'
    END = '\033[0m'

def print_highlighted_prediction(predicted):
    print('Sense: ', predicted['Sense'], '\t', 'Type: ', predicted['Type'])
    ret = ''
    docid = predicted['DocID']
    relation_sent_id = sorted(list(set([predicted['Arg1_sent_index'], predicted['Arg2_sent_index']])))
    arg1_token_id = [(predicted['Arg1_sent_index'], o) for o in predicted['Arg1']['TokenList']]
    arg2_token_id = [(predicted['Arg2_sent_index'], o) for o in predicted['Arg2']['TokenList']]
    conn_token_id = [(predicted['conn_sent_offset'][0], o) for o in predicted['Connective']['TokenList']] if predicted['Type'] in ['Explicit', 'AltLex', 'AltLexC'] else []
    for sent_id in relation_sent_id:
        sent = pdtb.get_sent_words(docid, sent_id)
        for token_id, word in enumerate(sent):
            if check_if_arg(token_id, sent_id, arg1_token_id):
                ret += color.RED + word[0] + color.END + ' '
            elif check_if_arg(token_id, sent_id, arg2_token_id):
                ret += color.BLUE + word[0] + color.END + ' '
            elif check_if_arg(token_id, sent_id, conn_token_id):
                ret += color.GREEN + word[0] + color.END + ' '
            else:
                ret += word[0] + ' '
    return ret
    
def get_correlated_span(predicted):
    ret = []
    arg1_sent_id = predicted['Arg1_sent_index']
    arg2_sent_id = predicted['Arg2_sent_index']
    docid = predicted['DocID']
    for i in range(len(pdtb.parse_data)):
        if arg1_sent_id == pdtb.get_arg_sent_id(i, 'Arg1')[0]:
            if arg2_sent_id == pdtb.get_arg_sent_id(i, 'Arg2')[0]:
                if docid == pdtb.parse_data[i]['DocID']:
                    ret.append(i)
    return ret

In [210]:
i = 6
predicted = output[i]

docid = predicted['DocID']
arg1_sent_id = predicted['Arg1_sent_index']
arg2_sent_id = predicted['Arg2_sent_index']
token_minus_offset_arg1 = get_minus_offset(pdtb.parse_dict, docid, arg1_sent_id)
token_minus_offset_arg2 = get_minus_offset(pdtb.parse_dict, docid, arg2_sent_id)
predicted['Arg1']['TokenList'] = [o-token_minus_offset_arg1 for o in predicted['Arg1']['TokenList']]
predicted['Arg2']['TokenList'] = [o-token_minus_offset_arg2 for o in predicted['Arg2']['TokenList']]
if predicted['Type'] in ['Explicit', 'AltLex', 'AltLexC']:
    predicted['conn_sent_offset'][0] = arg2_sent_id
    conn_sent_id = predicted['conn_sent_offset'][0]
    token_minus_offset_conn = get_minus_offset(pdtb.parse_dict, docid, conn_sent_id)
#     print(token_minus_offset_conn)
    predicted['Connective']['TokenList'] = [o-token_minus_offset_conn for o in predicted['Connective']['TokenList']]
    print(predicted['Connective']['TokenList'])
    
print(print_highlighted_prediction(predicted))


correlated_span = get_correlated_span(predicted)


print('\n')
print('--------- gold relation -----------')
print('\n')

for c in correlated_span:
    print(pdtb.get_highlighted_relation(c, v=True))
    print('\n')

[8]
Sense:  ['Temporal.Synchronous'] 	 Type:  Explicit
On the exchange floor , `` as soon as UAL stopped trading , we braced for a panic , '' said one top floor trader . 


--------- gold relation -----------


Relation: 46376,  Green for Connective(if any), Red for Arg1 and Blue for Arg2. 
DocID: wsj_2300,	Type: Explicit,	Sense: Temporal.Asynchronous.Succession,
Arg1_sent_id: 24	,	Arg2_sent_id: 24
On the exchange floor as soon as UAL stopped trading we braced for a panic 




In [211]:
predicted

{'ID': 6,
 'DocID': 'wsj_2300',
 'Arg1': {'TokenList': [6, 7]},
 'Arg2': {'TokenList': [9, 10, 11]},
 'Type': 'Explicit',
 'Sense': ['Temporal.Synchronous'],
 'Connective': {'TokenList': [8]},
 'Arg1_sent_index': 24,
 'Arg2_sent_index': 24,
 'conn_sent_offset': [24],
 'conn_name': 'as'}